In [35]:
import json

In [36]:
with open("qpsr.json") as f:
    data = json.load(f)

In [37]:
res = {}
for item in data:
    if not "pages" in item:
        print(f"Missing pages: {item['title']} ({item['year']})")
        pages = "?-?"
    else:
        pages = item["pages"]
    year = item["year"]
    if not "volume" in item:
        print(f"Missing volume: {item['title']} ({item['year']})")
        vol = "??"
    else:
        vol = item["volume"]
    if not "edition" in item:
        print(f"Missing edition: {item['title']} ({item['year']})")
        ed = "??"
    else:
        ed = item["edition"]
    if not "-" in pages:
        if pages.isdigit():
            start = end = pages
        else:
            raise IOError(f"No '-' in pages: {pages} ({key})")
    else:
        start, end = pages.split("-")
    if item["title"] == ".":
        print(item["pdf"])
    key = f"{year}_{vol}_{ed}"
    if not key in res:
        res[key] = [{"start": start, "end": end}]
    else:
        res[key].append({"start": start, "end": end})